In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.optimize import bisect
from scipy.sparse.linalg import eigs
from numba import njit

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 12, 'figure.figsize': (10, 6), 'figure.dpi': 130,
                     'axes.titlesize': 'x-large', 'axes.labelsize': 'large',
                     'xtick.labelsize': 'medium', 'ytick.labelsize': 'medium'})
np.set_printoptions(suppress=True, linewidth=120, precision=4)

# --- Utility Functions ---
def note(msg, **kwargs):
    display(Markdown(f"<div class='alert alert-block alert-info'>📝 **Note:** {msg}</div>"))
def sec(title):
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized for Heterogeneous Agent New Keynesian (HANK) model analysis.")

# Part 4: Macroeconomic Models
## Chapter 4.10: Heterogeneous Agent New Keynesian (HANK) Models

### Table of Contents
1.  [Introduction: The HANK Revolution](#1.-Introduction:-The-HANK-Revolution)
2.  [The Canonical HANK Model Environment](#2.-The-Canonical-HANK-Model-Environment)
3.  [Solving for the Stationary General Equilibrium](#3.-Solving-for-the-Stationary-General-Equilibrium)
    *   [3.1 The Aiyagari Model as the HANK Backbone](#3.1-The-Aiyagari-Model-as-the-HANK-Backbone)
    *   [3.2 The Full GE Solution Algorithm](#3.2-The-Full-GE-Solution-Algorithm)
4.  [Policy Transmission in HANK Models](#4.-Policy-Transmission-in-HANK-Models)
    *   [4.1 The Importance of the MPC Distribution](#4.1-The-Importance-of-the-MPC-Distribution)
    *   [4.2 The Indirect Effects of Monetary Policy](#4.2-The-Indirect-Effects-of-Monetary-Policy)
5.  [Chapter Summary](#5.-Chapter-Summary)
6.  [Exercises](#6.-Exercises)

### 1. Introduction: The HANK Revolution
This chapter explores the frontier of modern macroeconomic research: **Heterogeneous Agent New Keynesian (HANK)** models. These models represent a major leap forward by combining the rich household heterogeneity from the Aiyagari/Krusell-Smith tradition with the nominal rigidities and monetary policy framework of New Keynesian economics.

The motivation is simple: the real world is populated by diverse households, and this diversity matters for policy. The **Marginal Propensity to Consume (MPC)**—the fraction of an extra dollar of income that is spent—is not a single number, but a distribution. It is high for poor, credit-constrained households and low for wealthy households. HANK models, by explicitly modeling this distribution, provide a much more realistic and nuanced analysis of how monetary and fiscal policies affect the economy.

They show that the traditional view of monetary policy working through intertemporal substitution is incomplete. Instead, the most powerful effects work indirectly through general equilibrium effects on labor income, which are then amplified by the high MPCs of a large fraction of the population. This has profound implications for how we think about the role and power of both monetary and fiscal stabilization policy.

### 2. TANK Models: A Tractable Introduction to Heterogeneity

Before diving into the full HANK framework, we start with the **Two-Agent New Keynesian (TANK)** model, developed by Galí, López-Salido, and Vallés (2007). It provides a tractable way to introduce a simple but powerful form of heterogeneity.

The model splits the population into two types:
1.  **Ricardian Households**: These are the standard, forward-looking agents who save and borrow to smooth consumption.
2.  **Hand-to-Mouth (HtM) Households**: These agents simply consume their current disposable income in every period.

This simple division has profound implications for the transmission of monetary and fiscal policy. TANK models are a vital stepping stone toward the full-blown HANK models.

#### 2.1 The TANK IS Curve

The key innovation is the aggregate IS curve. With a fraction $\lambda$ of HtM households, aggregate consumption is $C_t = (1-\lambda) C_{R,t} + \lambda C_{HtM, t}$. Combining the Euler equation for Ricardian agents with the simple rule for HtM agents ($C_{HtM, t} = Y_t - T_t$) yields the TANK IS curve:
$$ x_t = E_t[x_{t+1}] - \frac{1}{\sigma_{eff}}(\hat{i}_t - E_t[\hat{\pi}_{t+1}]) \quad \text{where} \quad \sigma_{eff} = \frac{\sigma}{1-\lambda} $$
The presence of HtM agents effectively makes the economy behave as if it has a **lower intertemporal elasticity of substitution**. Intuitively, aggregate demand becomes more sensitive to current income (via the HtM agents) and less sensitive to interest rates and future income.

#### 2.2 Policy Implications of TANK Models

- **Monetary Policy is Dampened:** The standard channel of monetary policy works through intertemporal substitution. Since HtM households do not engage in this, the larger their share $\lambda$, the weaker this channel becomes.
- **Fiscal Policy is Amplified:** Ricardian households are subject to Ricardian Equivalence and will save a tax cut. HtM households will consume it immediately. The aggregate consumption response to a tax cut $T_t$ is $\Delta C_t = \lambda T_t$, making fiscal policy more potent.

### 3. The Canonical HANK Model Environment
A full HANK model is a complex machine with many moving parts. Here we outline the key components of a simplified but representative version.

- **Households**: A continuum of households face idiosyncratic income risk and can save in a single risk-free asset (which in equilibrium will be physical capital and government bonds). They face a borrowing constraint. This is the Aiyagari model backbone.

- **Firms**: A continuum of monopolistically competitive firms that produce differentiated goods and face Calvo-style price-setting frictions. This generates a **New Keynesian Phillips Curve (NKPC)**.

- **Central Bank**: Sets the nominal interest rate on government bonds according to a **Taylor rule**.

- **Government**: Issues bonds to finance its expenditures and potentially makes transfer payments. It must satisfy an intertemporal budget constraint.

**General Equilibrium**: The model is closed by finding a set of prices (wages, interest rates) and policy functions such that all agents are optimizing, markets clear, and the government budget constraint holds. A key difference from TANK models is that the interest rate directly affects the returns for *all* households who are saving, creating a richer feedback loop.

### 3. Solving for the Stationary General Equilibrium

#### 3.1 The Aiyagari Model as the HANK Backbone
The core of the HANK model is a general equilibrium Aiyagari model. Solving for the stationary equilibrium requires finding the market-clearing real interest rate $r$.

In [ ]:
sec("Aiyagari Model: General Equilibrium Solver")

class AiyagariSolver:
    def __init__(self, beta=0.96, sigma=3.0, alpha=0.33, delta=0.05, 
                 a_size=200, a_max=35, y_vals=None, P=None):
        self.beta, self.sigma, self.alpha, self.delta = beta, sigma, alpha, delta
        self.a_vals = np.linspace(1e-5, a_max, a_size)
        if y_vals is None:
            self.y_vals = np.array([0.5, 1.0, 1.5]); self.P = np.array([[0.8, 0.1, 0.1], [0.1, 0.8, 0.1], [0.1, 0.1, 0.8]])
        else: self.y_vals, self.P = np.array(y_vals), np.array(P)
        self.u = lambda c: (c**(1 - self.sigma) - 1) / (1 - self.sigma)

    def solve_household_problem(self, r):
        V = np.zeros((len(self.y_vals), len(self.a_vals)))
        policy_idx = np.zeros_like(V, dtype=np.int32)
        for i in range(1000):
            V_old = V.copy()
            EV = self.P @ V
            for y_idx, y in enumerate(self.y_vals):
                c = (1 + r) * self.a_vals + y - self.a_vals[:, np.newaxis]
                c = np.maximum(c, 1e-9)
                value_matrix = self.u(c) + self.beta * EV[y_idx, :]
                V[y_idx, :], policy_idx[y_idx, :] = np.max(value_matrix, axis=0), np.argmax(value_matrix, axis=0)
            if np.max(np.abs(V - V_old)) < 1e-7: break
        return policy_idx, V

    def get_stationary_distribution(self, policy_idx):
        n_a, n_y = len(self.a_vals), len(self.y_vals)
        T = np.zeros((n_y * n_a, n_y * n_a))
        for s_idx in range(n_y * n_a):
            y_idx, a_idx = s_idx // n_a, s_idx % n_a
            a_prime_idx = policy_idx[y_idx, a_idx]
            for y_prime_idx in range(n_y):
                s_prime_idx = y_prime_idx * n_a + a_prime_idx
                T[s_idx, s_prime_idx] += self.P[y_idx, y_prime_idx]
        vals, vecs = eigs(T.T, k=1, which='LM')
        dist_flat = vecs[:, 0].real
        return (dist_flat / dist_flat.sum()).reshape(n_y, n_a)

    def solve_ge(self):
        asset_supply = lambda r: (self.alpha / (r + self.delta))**(1 / (1 - self.alpha))
        def excess_demand(r):
            policy_idx, _ = self.solve_household_problem(r)
            dist = self.get_stationary_distribution(policy_idx)
            asset_demand = np.sum(dist * self.a_vals)
            return asset_demand - asset_supply(r)
        r_star = bisect(excess_demand, -self.delta + 0.001, 1/self.beta - 1 - 0.001)
        return r_star

note("Aiyagari general equilibrium solver class defined.")

#### 3.2 The Full GE Solution Algorithm
Solving a HANK model involves finding a fixed point in the mapping from prices to household decisions and back to aggregate quantities and prices. A simplified algorithm for the stationary equilibrium is:
1.  **Guess** an interest rate $r$.
2.  **Solve Household Problem:** Given $r$ and the implied wage $w(r)$, solve the household's DP problem to get their policy function for savings, $a'(a,y; r)$.
3.  **Find Stationary Distribution:** Compute the stationary distribution $\Psi(a,y)$ consistent with the policy function.
4.  **Aggregate:** Calculate the aggregate demand for capital from the distribution: $K^D = \int a \, d\Psi(a,y)$.
5.  **Check Market Clearing:** Compare $K^D$ to the capital supply from the firm's problem, $K^S(r)$. If they don't match, update the guess for $r$ (e.g., using a bisection method) and repeat.

#### 3.3 Solving for Dynamics: The Sequence-Space Jacobian Method

While the previous algorithm finds the steady state, modern HANK models are often solved using a more efficient and flexible method for computing dynamics: solving for the **sequence-space Jacobian**. This method, popularized by Auclert et al. (2021), directly computes the general equilibrium impulse responses to a shock without needing to iterate on beliefs.

The core idea is to treat the entire time path of a variable (its "sequence") as a single object. We then compute the Jacobians that map the sequence of shocks to the sequence of endogenous outcomes.

**Key Steps:**
1.  **Define the model's equations:** This includes the household's problem (Bellman equation), the firm's problem, market clearing conditions, and the central bank's policy rule.
2.  **Solve for the steady state:** Find the long-run equilibrium of the model where all variables are constant.
3.  **Linearize the model:** Linearize the dynamic equations of the model around the steady state.
4.  **Compute Jacobians:** This is the heart of the method. We need to compute how a small change in an aggregate variable at one point in time affects other variables at all points in time. For example, how does a small, one-time shock to the interest rate affect the entire time path of aggregate consumption? This is done using numerical differentiation.
5.  **Solve the linear system:** Once we have the Jacobians, we can solve for the economy's response to any shock by solving a large system of linear equations.

This method is powerful because it can handle complex models and is much more efficient than trying to solve the full non-linear model with time iteration.

In [ ]:
sec("Solving for the Aiyagari Stationary Equilibrium")
aiyagari_solver = AiyagariSolver()
note("Solving for the general equilibrium... (this may take a moment)")
r_star = aiyagari_solver.solve_ge()
note(f"Found market-clearing interest rate r* = {r_star:.4f}")

# --- Analyze the resulting distribution ---
policy_star_idx, _ = aiyagari_solver.solve_household_problem(r_star)
dist_star = aiyagari_solver.get_stationary_distribution(policy_star_idx)
wealth_dist = np.sum(dist_star, axis=0)

fig, ax = plt.subplots()
ax.bar(aiyagari_solver.a_vals, wealth_dist, width=aiyagari_solver.a_vals[1]-aiyagari_solver.a_vals[0])
ax.set_title('Figure 1: Stationary Wealth Distribution'); ax.set_xlabel('Assets'); ax.set_ylabel('Mass of Households')
plt.show()

### 4. Policy Transmission in HANK Models

#### 4.1 The Importance of the MPC Distribution
The distribution of the **Marginal Propensity to Consume (MPC)** is the key object for understanding policy transmission in HANK models. The MPC is highest for poor, borrowing-constrained households. Any policy that redistributes resources towards these high-MPC households will have a larger effect on aggregate demand.

#### 4.2 The Indirect Effects of Monetary Policy
In HANK models, the direct effect of interest rate changes on consumption (the intertemporal substitution channel) is weak because many households are constrained. However, monetary policy has powerful **indirect effects** that work through general equilibrium:
1.  A monetary easing lowers the real interest rate.
2.  This stimulates firm investment, increasing their demand for labor.
3.  Higher labor demand pushes up wages and reduces unemployment.
4.  This increase in labor income disproportionately benefits the low-wealth, high-MPC households.
5.  These households spend most of their extra income, leading to a large increase in aggregate consumption.

This **indirect channel**, operating through the labor market and amplified by the MPC distribution, is the primary transmission mechanism for monetary policy in HANK models.

### 5. Chapter Summary
- **Aiyagari Model:** The foundational incomplete markets model where precautionary savings against idiosyncratic income risk generates a non-degenerate wealth distribution.
- **HANK Models:** The modern frontier, embedding rich household heterogeneity into a New Keynesian framework. They provide a more realistic account of monetary and fiscal policy transmission.
- **Distribution is Key:** The effectiveness of policy depends crucially on the joint distribution of wealth, income, and MPCs across households.
- **Transmission Channels:** In HANK models, the indirect effects of policy on labor income and cash flows to high-MPC households are often more powerful than the traditional intertemporal substitution channel.

### 6. Exercises

1.  **Risk Aversion and Wealth Inequality:** In the Aiyagari model, how would you expect the stationary wealth distribution to change if households become more risk-averse (a higher $\sigma$)? Would the Gini coefficient be higher or lower? Explain the economic intuition and then run the solver to verify.

2.  **Income Process:** How would the stationary wealth distribution change if income shocks were more persistent (a higher autocorrelation in the transition matrix `P`)? What if they were more volatile?

3.  **Krusell-Smith:** Why is the assumption that the law of motion for aggregate capital can be well-approximated by a simple log-linear rule a reasonable one? What features of the model make this likely to be accurate?

4.  **Fiscal Policy in HANK:** Why are transfer payments (like stimulus checks) predicted to be much more powerful in a HANK model than in a RANK model? Which specific features of the HANK model are most important for this result?

5.  **Two-Asset Model:** In a HANK model with liquid and illiquid assets, how would you expect a household's MPC out of a small windfall to depend on their holdings of *both* assets? Who would have the highest MPC?